In [1]:
import torch
from torch import nn

# 注意：原文中使用了 "from d2l import torch as d2l"，这里主要依赖 torch
# 定义二维互相关运算函数
def corr2d(X, K):
    """计算二维互相关运算"""
    h, w = K.shape
    Y = torch.zeros((X.shape[0] - h + 1, X.shape[1] - w + 1))
    for i in range(Y.shape[0]):
        for j in range(Y.shape[1]):
            Y[i, j] = (X[i:i + h, j:j + w] * K).sum()
    return Y

# 测试数据
X = torch.tensor([[0.0, 1.0, 2.0], [3.0, 4.0, 5.0], [6.0, 7.0, 8.0]])
K = torch.tensor([[0.0, 1.0], [2.0, 3.0]])
print(corr2d(X, K))

# 定义卷积层类
class Conv2D(nn.Module):
    def __init__(self, kernel_size):
        super().__init__()
        self.weight = nn.Parameter(torch.rand(kernel_size))
        self.bias = nn.Parameter(torch.zeros(1))

    def forward(self, x):
        return corr2d(x, self.weight) + self.bias

tensor([[19., 25.],
        [37., 43.]])


In [2]:
# 创建一个 6x8 的图像，中间为0（黑色），两侧为1（白色），模拟边界
X = torch.ones((6, 8))
X[:, 2:6] = 0
print("Input X:\n", X)

# 构造卷积核 K，用于检测水平相邻元素的差异（边缘检测）
K = torch.tensor([[1.0, -1.0]])

# 进行互相关运算
Y = corr2d(X, K)
print("Output Y (Edge Detection):\n", Y)

Input X:
 tensor([[1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.]])
Output Y (Edge Detection):
 tensor([[ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.]])


In [3]:
# 构造一个二维卷积层，它具有1个输出通道和形状为(1, 2)的卷积核
conv2d = nn.Conv2d(1, 1, kernel_size=(1, 2), bias=False)

# 这个二维卷积层使用四维输入和输出格式（批量大小、通道、高度、宽度）
# 其中批量大小和通道数均为1
X = X.reshape((1, 1, 6, 8))
Y = Y.reshape((1, 1, 6, 7))
lr = 3e-2  # 学习率

for i in range(10):
    Y_hat = conv2d(X)
    l = (Y_hat - Y) ** 2
    conv2d.zero_grad()
    l.sum().backward()
    # 迭代卷积核
    conv2d.weight.data[:] -= lr * conv2d.weight.grad
    if (i + 1) % 2 == 0:
        print(f'epoch {i+1}, loss {l.sum():.3f}')

# 查看学习到的卷积核权重
print("Learned kernel weights:\n", conv2d.weight.data.reshape((1, 2)))

epoch 2, loss 10.425
epoch 4, loss 2.824
epoch 6, loss 0.914
epoch 8, loss 0.334
epoch 10, loss 0.130
Learned kernel weights:
 tensor([[ 0.9508, -1.0238]])


In [4]:
# 定义一个辅助函数来计算卷积层的输出形状
def comp_conv2d(conv2d, X):
    # 这里的 (1, 1) 表示批量大小和通道数都是1
    X = X.reshape((1, 1) + X.shape)
    Y = conv2d(X)
    # 省略前两个维度：批量大小和通道
    return Y.reshape(Y.shape[2:])

# 示例1：上下左右各填充1行/列
# 请注意，这里每边都填充了1行或1列，因此总共添加了2行或2列
conv2d = nn.Conv2d(1, 1, kernel_size=3, padding=1)
X = torch.rand(size=(8, 8))
print("Padding=1 shape:", comp_conv2d(conv2d, X).shape) # 应该是 (8, 8)

# 示例2：不对称填充
# 使用高为5、宽为3的卷积核。高度填充2，宽度填充1
conv2d = nn.Conv2d(1, 1, kernel_size=(5, 3), padding=(2, 1))
print("Asymmetric padding shape:", comp_conv2d(conv2d, X).shape) # 应该是 (8, 8)

Padding=1 shape: torch.Size([8, 8])
Asymmetric padding shape: torch.Size([8, 8])


In [5]:
# 示例1：将高度和宽度的步幅设置为2
conv2d = nn.Conv2d(1, 1, kernel_size=3, padding=1, stride=2)
print("Stride=2 shape:", comp_conv2d(conv2d, X).shape) # 应该是 (4, 4)

# 示例2：稍微复杂的例子
# padding=(0, 1), stride=(3, 4)
conv2d = nn.Conv2d(1, 1, kernel_size=(3, 5), padding=(0, 1), stride=(3, 4))
print("Complex stride shape:", comp_conv2d(conv2d, X).shape) # 应该是 (2, 2)

Stride=2 shape: torch.Size([4, 4])
Complex stride shape: torch.Size([2, 2])


In [6]:
# 多输入通道互相关运算
def corr2d_multi_in(X, K):
    # 先遍历 "X" 和 "K" 的第0个维度（通道维度），再把它们加在一起
    # 注意：这里的 corr2d 指的是上面第一部分定义的函数
    return sum(corr2d(x, k) for x, k in zip(X, K))

# 测试多输入通道
X_multi = torch.tensor([[[0.0, 1.0, 2.0], [3.0, 4.0, 5.0], [6.0, 7.0, 8.0]],
                        [[1.0, 2.0, 3.0], [4.0, 5.0, 6.0], [7.0, 8.0, 9.0]]])
K_multi = torch.tensor([[[0.0, 1.0], [2.0, 3.0]], 
                        [[1.0, 2.0], [3.0, 4.0]]])

print("Multi-channel input result:\n", corr2d_multi_in(X_multi, K_multi))

# 多输出通道互相关运算
def corr2d_multi_in_out(X, K):
    # 迭代 "K" 的第0个维度，每次都对输入 "X" 执行互相关运算。
    # 最后将所有结果都叠加在一起
    return torch.stack([corr2d_multi_in(X, k) for k in K], 0)

# 构造多输出通道的卷积核 (stack 3个卷积核)
K_multi_out = torch.stack((K_multi, K_multi + 1, K_multi + 2), 0)
print("K_multi_out shape:", K_multi_out.shape)

print("Multi-channel in/out result:\n", corr2d_multi_in_out(X_multi, K_multi_out))

Multi-channel input result:
 tensor([[ 56.,  72.],
        [104., 120.]])
K_multi_out shape: torch.Size([3, 2, 2, 2])
Multi-channel in/out result:
 tensor([[[ 56.,  72.],
         [104., 120.]],

        [[ 76., 100.],
         [148., 172.]],

        [[ 96., 128.],
         [192., 224.]]])


In [7]:
# 手动实现池化层
def pool2d(X, pool_size, mode='max'):
    p_h, p_w = pool_size
    Y = torch.zeros((X.shape[0] - p_h + 1, X.shape[1] - p_w + 1))
    for i in range(Y.shape[0]):
        for j in range(Y.shape[1]):
            if mode == 'max':
                Y[i, j] = X[i: i + p_h, j: j + p_w].max()
            elif mode == 'avg':
                Y[i, j] = X[i: i + p_h, j: j + p_w].mean()
    return Y

X_pool = torch.tensor([[0.0, 1.0, 2.0], [3.0, 4.0, 5.0], [6.0, 7.0, 8.0]])
print("Max pooling:\n", pool2d(X_pool, (2, 2)))
print("Avg pooling:\n", pool2d(X_pool, (2, 2), 'avg'))

# 使用 PyTorch 内置的 MaxPool2d
X_torch = torch.arange(16, dtype=torch.float32).reshape((1, 1, 4, 4))

# 默认 stride 与 kernel_size 相同
pool2d_layer = nn.MaxPool2d(3)
print("PyTorch MaxPool (size 3):\n", pool2d_layer(X_torch))

# 自定义 kernel size, stride 和 padding
# 矩形汇聚窗口 (2, 3)，stride (2, 3)，padding (0, 1)
pool2d_custom = nn.MaxPool2d((2, 3), stride=(2, 3), padding=(0, 1))
print("Custom PyTorch MaxPool:\n", pool2d_custom(X_torch))

Max pooling:
 tensor([[4., 5.],
        [7., 8.]])
Avg pooling:
 tensor([[2., 3.],
        [5., 6.]])
PyTorch MaxPool (size 3):
 tensor([[[[10.]]]])
Custom PyTorch MaxPool:
 tensor([[[[ 5.,  7.],
          [13., 15.]]]])
